<a href="https://colab.research.google.com/github/antonpetkoff/identifier-suggestion/blob/master/notebooks/00-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import utilities
import os
import shutil
from subprocess import check_output

In [2]:
!nvidia-smi

Sun Jun 14 14:32:34 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     8W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
%env WORKSPACE_DIR=/content/gdrive/My Drive/src

# TODO: how can one read an environment variable?!?
%cd '/content/gdrive/My Drive/src'

env: WORKSPACE_DIR=/content/gdrive/My Drive/src
/content/gdrive/My Drive/src


In [0]:
timestamp = check_output(['date', '-u', "+%Y-%m-%dT%H-%M-%S"]).decode('utf-8').strip()

os.environ['PROJECT_DIR'] = os.path.join(
    os.environ['WORKSPACE_DIR'],
    f'identifier-suggestion-{timestamp}',
)

In [6]:
!git clone https://github.com/antonpetkoff/identifier-suggestion.git --depth 1 "${PROJECT_DIR}"

Cloning into '/content/gdrive/My Drive/src/identifier-suggestion-2020-06-14T14-32-36'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 100 (delta 1), reused 53 (delta 1), pack-reused 0
Receiving objects: 100% (100/100), 316.99 KiB | 454.00 KiB/s, done.
Resolving deltas: 100% (1/1), done.


In [0]:
os.chdir(os.environ['PROJECT_DIR'])

In [8]:
!pwd
!ls -l

/content/gdrive/My Drive/src/identifier-suggestion-2020-06-14T14-32-36
total 65
drwx------  4 root root  4096 Jun 14 14:32 data
-rw-------  1 root root 35149 Jun 14 14:32 LICENSE
drwx------  2 root root  4096 Jun 14 14:32 notebooks
-rw-------  1 root root  9927 Jun 14 14:32 README.md
drwx------  2 root root  4096 Jun 14 14:32 requirements
drwx------ 13 root root  4096 Jun 14 14:32 src
drwx------  3 root root  4096 Jun 14 14:32 vscode-extension


In [9]:
%tensorflow_version 2.x

import tensorflow
print(tensorflow.__version__)

2.2.0


In [10]:
# Google Colab has standard libraries like numpy, pandas, matplotlib and TF (of course) pre-installed
!pip install -r requirements/colab.txt

In [11]:
# provide secrets to the project, e.g. access to wandb
shutil.copy(
    os.path.join(os.environ['WORKSPACE_DIR'], 'secrets/.env'),
    os.environ['PROJECT_DIR']
)

'/content/gdrive/My Drive/src/identifier-suggestion-2020-06-14T14-32-36/.env'

In [0]:
# monkey-patch (mock) os.symlink to be a noop, because wandb.save() uses it, but it is not supported by Google Colab Notebooks
os.symlink = lambda *x: print('Executing mocked noop symlink with arguments', x)

In [13]:
from argparse import Namespace
from src.pipelines.baseline import run

params = {
  'dir_data': '../data/processed/subtoken/',
  'file_checkpoint_dir': 'models/checkpoints/baseline/',
  'dir_preprocessed_data': '../data/processed/seq2seq/',
  'max_input_length': 200,
  'max_output_length': 8,
  'input_vocab_size': 20000,
  'input_embedding_dim': 50,
  'output_vocab_size': 16000,
  'output_embedding_dim': 50,
  'latent_dim': 320,
  'learning_rate': 0.0001,
  'epochs': 50,
  'batch_size': 256,
  'random_seed': 1,
  'eval_averaging': 'macro'
}

run(Namespace(**params))

Initializing logger


wandb: Wandb version 0.9.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Experiment parameters:  Namespace(batch_size=256, dir_data='../data/processed/subtoken/', dir_preprocessed_data='../data/processed/seq2seq/', epochs=50, eval_averaging='macro', file_checkpoint_dir='models/checkpoints/baseline/', input_embedding_dim=50, input_vocab_size=20000, latent_dim=320, learning_rate=0.0001, max_input_length=200, max_output_length=8, output_embedding_dim=50, output_vocab_size=16000, random_seed=1)
Loading preprocessed files...
Loaded input vocabulary.
Loaded output vocabulary.
Loaded preprocessed files.
Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
EncoderEmbedding (Embedding) multiple                  1000000   
_________________________________________________________________
EncoderLSTM (LSTM)           multiple                  474880    
Total params: 1,474,880
Trainable params: 1,474,880
Non-trainable params: 0
______________________________________________

wandb: Visualizing heatmap.


epoch 1 evaluation time: 51.03440523147583 sec
epoch evaluation:  {'epoch': 1, 'epoch_test_loss': <tf.Tensor: shape=(), dtype=float32, numpy=2.6472056>, 'epoch_test_sparse_categorical_accuracy': <tf.Tensor: shape=(), dtype=float32, numpy=0.18958901>, 'epoch_test_precision': <tf.Tensor: shape=(), dtype=float32, numpy=0.0012924408>, 'epoch_test_recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.001259662>, 'epoch_test_f1': <tf.Tensor: shape=(), dtype=float32, numpy=0.000962027>}


...

epoch 27 - batch 10 - loss 0.9668779373168945 - precision 0.0713680163025856 - recall 0.0667765662074089 - f1 0.06629811972379684 - sparse_categorical_accuracy 0.36462053656578064
epoch 27 - batch 20 - loss 0.9428107142448425 - precision 0.10413233190774918 - recall 0.09500408917665482 - f1 0.09545750170946121 - sparse_categorical_accuracy 0.36503905057907104
epoch 27 - batch 30 - loss 1.143860936164856 - precision 0.12797309458255768 - recall 0.11628252267837524 - f1 0.11690866202116013 - sparse_categorical_accuracy 0.3648437559604645
epoch 27 - batch 40 - loss 0.8899162411689758 - precision 0.14768683910369873 - recall 0.13325026631355286 - f1 0.13450203835964203 - sparse_categorical_accuracy 0.36329519748687744
epoch 27 - batch 50 - loss 1.004158616065979 - precision 0.16518442332744598 - recall 0.14952339231967926 - f1 0.15098837018013 - sparse_categorical_accuracy 0.3641406297683716
epoch 27 - batch 60 - loss 1.0450865030288696 - precision 0.1789679080247879 - recall 0.16120870411

wandb: Visualizing heatmap.


epoch 27 evaluation time: 51.39547848701477 sec
epoch evaluation:  {'epoch': 27, 'epoch_test_loss': <tf.Tensor: shape=(), dtype=float32, numpy=1.4310726>, 'epoch_test_sparse_categorical_accuracy': <tf.Tensor: shape=(), dtype=float32, numpy=0.319932>, 'epoch_test_precision': <tf.Tensor: shape=(), dtype=float32, numpy=0.20734774>, 'epoch_test_recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.16866793>, 'epoch_test_f1': <tf.Tensor: shape=(), dtype=float32, numpy=0.17430505>}


wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.


ValueError: ignored